In [22]:
import wandb
import os

In [23]:
RUN_PATH = "team-epoch-iv/detect-harmful-brain-activity/wkg0bfwe"

In [24]:
api = wandb.Api()
run = api.run(f"{RUN_PATH}")

In [25]:
# list all artefacts
model_artefacts = []
for artefact in run.logged_artifacts():
    if artefact.type == "model":
        print(artefact.name)
        model_artefacts.append(artefact)

EEGNet:v1695
EEGNet:v1696
EEGNet:v1697
EEGNet:v1700
EEGNet:v1703
EEGNet:v1704
EEGNet:v1705
EEGNet:v1706
EEGNet:v1707
EEGNet:v1708


In [26]:
# download all artefacts
for artefact in model_artefacts:
    for file in artefact.files():
        print(f"Downloading {file.name} from {artefact.name}")
        file.download(root="../downloads")

In [27]:
# get the config type artefact (should be only one)
config_artefact = None
for artefact in run.logged_artifacts():
    if artefact.type == "config":
        config_artefact = artefact
        break

In [30]:
for file in config_artefact.files():
    if file.name not in ["config.yaml","cv.yaml","train.yaml"]:
        print(f"Downloading {file.name} from {config_artefact.name}")
        file.download(root="../downloads", replace=True)
        # rename the file to the name of the run
        os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}")
        print(f"Renamed to {run.name}")

Renamed to simmered-tin
